In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
cd "/gdrive/My Drive/haeun_textgen/"

/gdrive/My Drive/haeun_textgen


In [3]:
ls

data/  Generation/  main.ipynb


In [4]:
!pip install pytorch-lightning==1.6.1
!pip install transformers==4.28.1

In [6]:
import torch
from Generation.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_corpus_name="newyear", #카테고리마다 바꾸기
    downstream_model_dir="/gdrive/My Drive/haeun_textgen/checkpoint-generation",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [7]:
from Generation import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [8]:
nlpbook.set_logger(args)

INFO:umatter:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='newyear', downstream_corpus_root_dir='/gdrive/My Drive/haeun_textgen/data', downstream_model_dir='/gdrive/My Drive/haeun_textgen/checkpoint-generation', max_seq_length=130, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:umatter:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='newyear', downstream_corpus_root_dir='/gdrive/My Drive/haeun_textgen/data', downstream_model_dir='/gdrive/My Drive/haeun_textgen/checkpoint-generation', max_seq_length=130, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mo

In [9]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [10]:
from Generation.nlpbook.generation import CustomCorpus, GenerationDataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = CustomCorpus()
train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:umatter:Loading features from cached file /gdrive/My Drive/haeun_textgen/data/newyear/cached_train_PreTrainedTokenizerFast_130_newyear_sentence-generation [took 0.568 s]
INFO:umatter:Loading features from cached file /gdrive/My Drive/haeun_textgen/data/newyear/cached_train_PreTrainedTokenizerFast_130_newyear_sentence-generation [took 0.568 s]


In [11]:
val_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:umatter:Loading features from cached file /gdrive/My Drive/haeun_textgen/data/newyear/cached_test_PreTrainedTokenizerFast_130_newyear_sentence-generation [took 0.360 s]
INFO:umatter:Loading features from cached file /gdrive/My Drive/haeun_textgen/data/newyear/cached_test_PreTrainedTokenizerFast_130_newyear_sentence-generation [took 0.360 s]


In [12]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name
)

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [13]:
from Generation.nlpbook.generation import GenerationTask
task = GenerationTask(model, args)

In [14]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [15]:
# approx. 2 mins
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py:1927: Poss

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [16]:
def inference_fn(
        prompt,
        num_sentences=20,  # Change to the number of sentences you want to generate
        min_length=10,
        max_length=50,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0,
        temperature=1.0,
):
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        with torch.no_grad():
            generated_sentences = []
            for _ in range(num_sentences):
                generated_ids = model.generate(
                    input_ids,
                    do_sample=True,
                    top_p=float(top_p),
                    top_k=int(top_k),
                    min_length=int(min_length),
                    max_length=int(max_length),
                    repetition_penalty=float(repetition_penalty),
                    no_repeat_ngram_size=int(no_repeat_ngram_size),
                    temperature=float(temperature),
                )
                generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])
                generated_sentences.append(generated_sentence)
    except:
        generated_sentences = ["처리 중 오류가 발생했습니다. <br>"
                               "변수의 입력 범위를 확인하세요. <br><br>"
                               "min_length: 1 이상의 정수 <br>"
                               "max_length: 1 이상의 정수 <br>"
                               "top-p: 0 이상 1 이하의 실수 <br>"
                               "top-k: 1 이상의 정수 <br>"
                               "repetition_penalty: 1 이상의 실수 <br>"
                               "no_repeat_ngram_size: 1 이상의 정수 <br>"
                               "temperature: 0 이상의 실수"
                               ]
    return {
        'results': generated_sentences,  # Return multiple generated sentences as a list
    }


In [17]:
#새해
newyear = inference_fn("새해 2024년")
newyear

{'results': ['새해 2024년 갑진년에는 더 의미있고 행복한 변화가 이뤄지길 바라겠습니다.</s>',
  '새해 2024년 모든 소망이 이루어지길 바랍니다. 오늘은 너의 날, 좋은 날이 되길 바랍니다.</s>',
  '새해 2024년 새해에는 여러분에게 특별한 모습으로 행복과 성취가 함께할 것입니다.</s>',
  '새해 2024년은 모든 소망이 이뤄지는 해가 되기를 바라요.</s>',
  '새해 2024년은 더 멋진 순간이 찾아오길 바랍니다!</s>',
  '새해 2024년 행복한 해! 당신의 날은 너에게 더 특별한 기쁨이 가득하길 바라며, 새해에는 당신의 곁으로 다가오길 잘해봐.</s>',
  '새해 2024년 새해에도 모든 소망이 이루어지길 기원합니다.</s>',
  '새해 2024년 더욱 열심히 보내봐. 너에게 미안한 일 너, 항상 고맙고 축하해! 올 해 더 많은 사랑과 기쁨이 찾아올 수 있길 바라! 너의 날, 즐거운 일들이 가득하시길 기원한다.</s>',
  '새해 2024년은 희망의 새해를 맞이하여 힘찬 시작의 해가 되길 기원합니다.</s>',
  '새해 2024년 갑진년 새해에는 모든 일들이 순조롭게 풀리면서 갑진년 새해가 당신의 목표임을 다시 한 번 되새겨~</s>',
  '새해 2024년 새해에는 희망과 행복이 넘치는 모든 순간들이 함께하길 기원합니다. 새해의 소망을 이루어드리시길 바랍니다.</s>',
  '새해 2024년 새해에 행복한 일들이 여러분을 맞이하기를 기원합니다.</s>',
  '새해 2024년 갑진년에는 한 해, 여러분과 함께 모든 일들이 행복한 해가 되기를 바라ᄒᄒᄒ</s>',
  '새해 2024년 새해 복 많이 받으세요.</s>',
  '새해 2024년 갑진년 새해가 행복과 행복으로 가득하길 기원합니다.</s>',
  '새해 2024년 새해에도 함께 행복하자. 새해에는 희망찬 새해가 되시길 바라.</s>',
  '새해 2024년 갑진년에는 좋은 결실을 이루는 해 올 한해는 우리 모두 행복하고 행복한 모습을 보여주는 